In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/attilaimre99/CPP1708/refs/heads/main/train.csv"
)
df.head()

,id,name,source,sequence,smiles,label,len,average_wt,is_cyclic
0,0,MLCPP2_non_CPP_4095,MLCPP2,NVQLGPSLTEKL,CC(C)C[C@H](NC(=O)[C@H](CCCCN)NC(=O)[C@H](CCC(...,0,12,1298.505,False
1,1,Dobchevetal_CPP_3291,Dobchevetal,RLIKTLKTLLQKRKTL,CC[C@H](C)[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@@H...,1,16,1953.499,False
2,2,C2Pred_CPP_145,C2Pred,WKCRRQCFRVLHHWN,CC(C)C[C@H](NC(=O)[C@@H](NC(=O)[C@H](CCCNC(=N)...,1,15,2069.466,False
3,3,MLCPP2_non_CPP_4717,MLCPP2,YHLSKEDAEVINATKKRGNKVI,CC[C@H](C)[C@H](NC(=O)[C@@H](NC(=O)[C@H](CCCCN...,0,22,2513.887,False
4,4,CellPPDindependent_CPP_1272,CellPPDindependent,RKKTFKEVANAVKISA,CC[C@H](C)[C@H](NC(=O)[C@H](CCCCN)NC(=O)[C@@H]...,1,16,1790.146,False
